<a href="https://colab.research.google.com/github/raheelam98/LangChain_Fundamentals/blob/main/langchain_course_jupyter/section_3_data_processing/part1_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Section 3**

#### Integrating Linkedin Data Processing

**Part 1 - Scraping**



In [12]:
# Install the required packages:
%%capture --no-stderr
%pip install -U  langsmith # check
%pip install --quiet -U  langchain_google_genai langchain_core langchain langgraph   python-dotenv langsmith

In [ ]:
# # packages:
# %%capture --no-stderr
# %pip install -U langgraph langsmith  langchain   python-dotenv


In [13]:
import os
from google.colab import userdata

os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "ice_breaker"

In [14]:
# API Keys
# Get the GEMINI API key from user data
from google.colab import userdata
gemini_api_key = userdata.get('GEMINI_API_KEY')

In [15]:
# Initialize the ChatGoogleGenerativeAI with the Gemini model

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",  # Specify the model to use
    max_retries=2,
    api_key=gemini_api_key    # Provide the Google API key for authentication
)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm: ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(model="gemini-1.5-flash", api_key=gemini_api_key)

In [22]:
import os
import requests


def scrape_linkedin_profile(linkedin_profile_url: str, mock: bool = False):
    """Scrape information from LinkedIn profiles.
    Manually scrape the information from the LinkedIn profile."""

    if mock:
        linkedin_profile_url = "https://gist.githubusercontent.com/emarco177/0d6a3f93dd06634d95e46a2782ed7490/raw/78233eb934aa9850b689471a604465b188e761a0/eden-marco.json"
        response = requests.get(
            linkedin_profile_url,
            timeout=10,
        )
    else:
        api_endpoint = "https://nubela.co/proxycurl/api/v1/linkedin/profile"
        header_dic = {"Authorization": f'Bearer {os.environ.get("PROXYCURL_API_KEY")}'}
        response = requests.get(
            api_endpoint,
            params={"url": linkedin_profile_url},
            headers=header_dic,
            timeout=10,
        )

    # Parse the response
    data = response.json()

    # Remove extra empty fields  (token:- Validate the payload of the token )
    data = {
        k: v
        for k, v in data.items()
        if v not in ([], "", None)
        and k not in ["people_also_viewed", "certifications"]
    }
    if data.get("groups"):
        for group_dict in data.get("groups"):
            group_dict.pop("profile_pic_url", None)

    return data


if __name__ == "__main__":
    print(
        scrape_linkedin_profile(
            linkedin_profile_url="https://www.linkedin.com/in/eden-marco/",
        )
    )


{'code': 404, 'description': 'The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.', 'name': 'Not Found'}


In [18]:
from langchain.prompts.prompt import PromptTemplate


print("Hello LangChain")

summary_template = """
    given the information {information} about a person I want you to create:
    1. A short summary
    2. two interesting facts about them
"""

summary_prompt_template = PromptTemplate(
        input_variables=["information"], template=summary_template
    )

chain = summary_prompt_template | llm


linkedin_data = scrape_linkedin_profile(linkedin_profile_url="https://www.linkedin.com/in/eden-marco/")

res = chain.invoke(input={"information": linkedin_data})
print(res)


Hello LangChain
content="It's impossible to create a person's summary and interesting facts based solely on an API error message.  The provided information describes an *error*, not a person.  The error indicates an authentication failure, not characteristics of an individual.  To create a person, I'd need information about their life, personality, etc., not an API response.\n" additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-a55a8066-28d3-427c-975a-f53f886d1468-0' usage_metadata={'input_tokens': 54, 'output_tokens': 74, 'total_tokens': 128, 'input_token_details': {'cache_read': 0}}
